In [6]:
import torch
import sys
sys.path.append("..")

import json, time, os
from statsmodels.tsa.arima.model import ARIMA
from utils import get_gnn_data
from log.logutli import Logger

In [2]:
x = torch.randn(64, 134, 288, 10)

print(x.shape)

torch.Size([64, 134, 288, 10])


In [22]:
with open('../config.json', 'r') as f:
    config = json.load(f)
config['data_path'] = '../data/'
config["location_path"] = "../data/sdwpf_baidukddcup2022_turb_location.CSV"


# "data_path"      : "./data/",
#     "filename"       : "wtbdata_245days.csv",
#     "location_path"  : "./data/sdwpf_baidukddcup2022_turb_location.CSV",

start_time   = time.time()
current_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(start_time))
save_dir     = os.path.join("../result", current_time + f'_test')
log_id       = 'main'
log_name     = f'Run_{current_time}.log'
log_level    = 'info'
Logger_      = Logger(log_id, save_dir, log_name, log_level)
logger       = Logger_.logger
logger.info(f"LOCAL TIME: {current_time}")

train_original_data, val_original_data, test_original_data, _, _, _ = get_gnn_data(config, logger)  

2024-01-08 17:32:09-main-INFO: LOCAL TIME: 2024_01_08_17_32_09


log_name: Run_2024_01_08_17_32_09.log


2024-01-08 17:32:14-main-INFO: Raw data.shape: (4727520, 13)
2024-01-08 17:32:15-main-INFO: data.shape: (134, 10, 35280), means.shape: (10,), stds.shape: (10,)
2024-01-08 17:32:15-main-INFO: train_original_data: (134, 10, 28800), 
val_original_data: (134, 10, 2880), 
test_original_data: (134, 10, 3600).


In [11]:
x1 = train_original_data[0, -1, :]
print(f'x1.shape: {x1.shape}')
input_time_step = 48
forecast_steps = 24

train, test = x1[:input_time_step], x1[input_time_step:input_time_step+forecast_steps]

# train_size = 80
# train, test = data[:train_size], data[train_size:]

# 训练ARIMA模型
order = (2, 1, 2)  # ARIMA(p, d, q)参数，可以根据实际情况调整
model = ARIMA(train, order=order)
model_fit = model.fit()

# 多步向前预测
forecast = model_fit.forecast(steps=forecast_steps)
print(forecast.shape)

x1.shape: (28800,)
(24,)


In [12]:
print(test)
print(forecast)

[2.79396946 2.79439029 2.75946172 2.69734311 2.71969366 2.79457732
 2.7460888  2.79417988 2.78728301 2.69851208 2.77968476 2.70279047
 2.7347265  2.72114317 2.5437182  2.19223488 2.03687991 1.43818463
 1.6110974  1.38177055 1.28972653 1.00132037 0.63980736 0.94687015]
[2.7936841  2.79360479 2.79356603 2.79354963 2.79354252 2.79353946
 2.79353814 2.79353757 2.79353732 2.79353722 2.79353717 2.79353715
 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714
 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714 2.79353714]


In [23]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

train_indices = [(i, i + (config['input_len'] + config['output_len'])) 
        for i in range(train_original_data.shape[2] - \
                        (config['input_len'] + config['output_len']) + 1)]
val_indices = [(i, i + (config['input_len'] + config['output_len'])) 
        for i in range(val_original_data.shape[2] - \
                        (config['input_len'] + config['output_len']) + 1)]

print(test_original_data.shape)


for turb_id in range(test_original_data.shape[0]):  # [num_nodes, num_features, seq_len]
    data = test_original_data[turb_id, -1, :]  # [seq_len, ]
    print(data.shape)
    for i in tqdm(range(test_original_data.shape[2] - \
                        (config['input_len'] + config['output_len']) + 1)):
        train, test = data[i:i+input_time_step], data[i+input_time_step:i+input_time_step+forecast_steps]

        # 训练ARIMA模型
        order = (1, 1, 1)  # ARIMA(p, d, q)参数，可以根据实际情况调整
        model = ARIMA(train, order=order)
        model_fit = model.fit()

        # 多步向前预测
        forecast = model_fit.forecast(steps=forecast_steps)
        # print(forecast.shape)
    if turb_id > 1:
        break 

(134, 10, 3600)
(3600,)


  0%|          | 0/3529 [00:00<?, ?it/s]

100%|██████████| 3529/3529 [02:51<00:00, 20.59it/s]


(3600,)


100%|██████████| 3529/3529 [02:58<00:00, 19.82it/s]


(3600,)


100%|██████████| 3529/3529 [03:00<00:00, 19.55it/s]


In [29]:
X = test_original_data[66, -1, :]
for i in tqdm(range(len(X) - \
                    (config['input_len'] + config['output_len']) + 1)):
    train, test = X[i:i+input_time_step], X[i+input_time_step:i+input_time_step+forecast_steps]

    order = (1, 1, 1)  # ARIMA(p, d, q)参数，可以根据实际情况调整
    model = ARIMA(X, order=order)
    model_fit = model.fit()

    # 多步向前预测
    forecast = model_fit.forecast(steps=forecast_steps)

100%|██████████| 3529/3529 [41:11<00:00,  1.43it/s]


In [27]:
X.shape

(3600,)